# Bayesian regression analysis in lmfit
In this example we will perform a Bayesian regression analysis in the `lmfit` package. You'll need the `emcee` and `corner` packages installed. Both of these are available on PyPI. We start off with the requisite imports.

In [ ]:
%matplotlib inline
import numpy as np
import lmfit
import matplotlib.pyplot as plt

Now lets create the data we'll then fit.

In [ ]:
x = np.linspace(1, 10, 250)
np.random.seed(0)
y = 3.0*np.exp(-x/2) - 5.0*np.exp(-(x-0.1) / 10.) + 0.1*np.random.randn(len(x))
plt.plot(x, y)

We'll need a `Parameters` object to store the Parameters of our model.

In [ ]:
p = lmfit.Parameters()
p.add_many(('a1', 4), ('a2', 4), ('t1', 3), ('t2', 3., True))

def residual(p):
    v = p.valuesdict()
    return v['a1']*np.exp(-x/v['t1']) + v['a2']*np.exp(-(x-0.1) / v['t2']) - y

We'll do a straightforward least-squares fit first, using the *Nelder-Mead* method.

In [ ]:
mi = lmfit.minimize(residual, p, method='Nelder', nan_policy='omit')
lmfit.printfuncs.report_fit(mi.params, min_correl=0.5)
plt.plot(x, y)
plt.plot(x, residual(mi.params) + y, 'r')

The above analysis ignored the uncertainty on each datapoint. So we'll add in a noise Parameter to address that. 

In [ ]:
# add a noise parameter
mi.params.add('noise', value=1, min=0.001, max=2)

For the Bayesian analysis we have to calculate the log-posterior probability for the system. The log-posterior probability is the sum of the log-prior and log-likelihood probabilities. 

The log-prior encodes prior beliefs about the system. With the `lmfit.emcee` method the log-prior probability is assumed to be 0 if a `Parameter` is within its bounds. If it's outside the bounds the log-prior probability is `-np.inf`, i.e. impossible. This is known as a uniform prior.

The log-likelihood function is the probability of the data given the model. The following function calculates the log-likelihood. Note how the data uncertainties (http://dan.iel.fm/emcee/current/user/line/) are included in the log-likelihood.

Note, if you want a non-uniform prior, then you should include the log-prior and log-likelihood in a single function.

In [ ]:
# This is the log-likelihood probability for the sampling. We're going to estimate the
# size of the uncertainties on the data as well.
def lnprob(p):
    noise = p['noise']
    return -0.5 * np.sum((residual(p) / noise)**2 + np.log(2 * np.pi * noise**2))

With `lmfit.emcee` we first have to create a `lmfit.Minimizer` object. We're going to initialise it with the best fit from above.

Then we do the sampling. We are going to burn in (discard) the first 300 steps out of a total of 1000. We are then going to thin, which reduces the amount of autocorrelation in the chain. Thinning by 20 means that 1 in 20 samples is kept.

In [ ]:
mini = lmfit.Minimizer(lnprob, mi.params)
res = mini.emcee(burn=300, steps=1000, thin=20, params=mi.params)

Now lets plot the outcome with the `corner` package. Note the banana shapes, they arise out of non-normality for the parameter probability distribution. The elliptical nature of several parameter pairs indicates a high degree of correlation. The correlation parameter is returned as an attribute of the `res` OptimizeResult.

In [ ]:
import corner
a=corner.corner(res.flatchain, labels=res.var_names, truths=list(res.params.valuesdict().values()))

Here are the output parameters, the correlations are also worked out. The `OptimizeResult` contains `chain` and `lnprob` attributes which contain the samples and corresponding log-probability. In addition, the Minimizer will then also contain a `sampler` attribute (a `emcee.EnsembleSampler` object) which can be used for other purposes.

In [ ]:
print("median of posterior probability distribution")
print('--------------------------------------------')
lmfit.report_fit(res.params)
# find the maximum likelihood solution
highest_prob = np.argmax(res.lnprob)
hp_loc = np.unravel_index(highest_prob, res.lnprob.shape)
mle_soln = res.chain[hp_loc]
for i, par in enumerate(p):
    p[par].value = mle_soln[i]
print("\nMaximum likelihood Estimation")
print('-----------------------------')
print(p)

In [ ]:
plt.plot(x, y)
plt.plot(x, residual(mi.params) + y, 'r', label='Nelder-Mead')
plt.plot(x, residual(res.params) + y, 'black', label='emcee')
plt.legend()

Finally, let's calculate some uncertainty intervals.

In [ ]:
quantiles = np.percentile(res.flatchain['t1'], [2.28, 15.9, 50, 84.2, 97.7])
print("1 sigma spread", 0.5 * (quantiles[3] - quantiles[1]))
print("2 sigma spread", 0.5 * (quantiles[4] - quantiles[0]))